In [1]:
## This code generates daily NDVI images from Sentinel-2 Data using Google Earth Engine (GEE) API

# Initializing GEE
import ee
# Trigger the authentication flow.
# Initialize the library.
ee.Authenticate()
ee.Initialize()

C:\Users\mirmazloumi\Anaconda3\envs\env\lib\site-packages\google\auth\crypt\_cryptography_rsa.py:22: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  import cryptography.exceptions


Enter verification code: 4/1AVHEtk5t7_bh0DSHBOx7__UY__Qb14r9nt1KbTPGs161watKr_JY2DHkodw

Successfully saved authorization token.


In [4]:
## Import Libraries

import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [5]:
## Functions

## Sentinel-2 Cloud Masking

def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    # bitwise left shift operator <<
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    # keep all the pixels where its bit 10 and 11 are both zeros and mask every pixel where this condition is not met.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
           .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000) \
                .select("B.*") \
                .copyProperties(image, ["system:time_start"])

## NDVI
def addmNDVI(image):
    ndvi = image.normalizedDifference(['B8_median', 'B4_median']).rename('mNDVI')
    return image.addBands(ndvi)

## Reading images name and date

def addNameAndDate(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    name = image.get('system:index')
    return image.set({'date': date, 'name': name})

# Generating Daily Mosaic
def day_mosaics(date, newlist):
    # Cast
    date = ee.Date(date)
    newlist = ee.List(newlist)

    # Filter collection between date and the next day
    filtered = S2Images.filterDate(date, date.advance(1,'day'))

    # Make the mosaic
    #image = filtered.mosaic() ## In case of no need to reducer.
    image = filtered.reduce(reducer)
    # Add the mosaic to a list only if the collection has images
    return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image.set('date', date).set('name', ee.String('Image_').cat(ee.Algorithms.String(date.format('YYYY-MM-dd'))))), newlist))


In [6]:
## Reading the shapefile from a local directory and changing it to a GEE-based geometry
shapefile_path_real = r'C:\Users\mirmazloumi\Nextcloud\01_data\Study_Area_vector\Busia\Busia_jsn.geojson'
gdf_r = gpd.read_file(shapefile_path_real)
geometry_r = gdf_r['geometry'][0]
Busia_shapefile = ee.Geometry(geometry_r.__geo_interface__)

In [9]:
## Number of Sentinel-2 images for particular dates

startDate = '2021-01-01'
endDate = '2022-01-01'

# Depending on the type of Sentinel-2 data, the collection can be selected.
# The region of interest
# The period of interest
# The minimum cloud cover in images
# Applying cloud masking function

S2Images = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filterBounds(Busia_shapefile)\
    .filterDate(startDate, endDate)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .map(maskS2clouds)\

## Getting the number of images
print(S2Images.size().getInfo())

134


In [17]:
## Generating Daily Mosaics

# If you get this error: "EEException: User memory limit exceeded.", you can change the finish date to half ...
# ... a year. So, finally you need to run the code from here for two times.
start = ee.Date.fromYMD(2021,1,1)
finish = ee.Date.fromYMD(2021,7,1)


# Difference in days between start and finish
diff = finish.difference(start, 'day')

# Make a list of all dates
range_list = ee.List.sequence(0, diff.subtract(1)).map(lambda day: start.advance(day,'day'))

## Since there are overlapped images over same pixels on each day, a reducer can get a value for it.
# Here, a meadian reducer is used:
reducer = ee.Reducer.median()


# Iterate over the range to make a new list, and then cast the list to an imagecollection
daily_col = ee.ImageCollection(ee.List(range_list.iterate(day_mosaics, ee.List([]))))

# Number of Images per day:
print(daily_col.size().getInfo())


27


In [12]:
# Generating daily NDVI maps
ndvi_col = daily_col.map(addmNDVI)
# Getting the ndvi layer
ndvi_only = ndvi_col.select('mNDVI')

In [13]:
## Making a GEE-based shapefile to crop the final output as the chosed shapefiles.
# Normally, the outputs are cropped as rectangle, but this code crops as the imported shapefile.

# Create a list to hold the polygon coordinates
polygons = []

# Loop over each geometry in the GeoDataFrame and extract the exterior coordinates
for polygon in gdf_r['geometry']:
    # Check if the geometry is a Polygon or a MultiPolygon
    if polygon.geom_type == 'Polygon':
        exterior_coords = []
        for coords in polygon.exterior.coords:
            exterior_coords.append(coords)
        polygons.append(exterior_coords)
    elif polygon.geom_type == 'MultiPolygon':
        for poly in polygon:
            exterior_coords = []
            for coords in poly.exterior.coords:
                exterior_coords.append(coords)
            polygons.append(exterior_coords)

# Create an EE polygon geometry using the extracted coordinates
Busia_shapefile_gee = ee.Geometry.Polygon(polygons)

C:\Users\mirmazloumi\Anaconda3\envs\env\lib\site-packages\ipykernel_launcher.py:16: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  app.launch_new_instance()


In [12]:
## Export NDVI images as TIFF file to Google Drive

# Set the export parameters
export_params = {
    'driveFolder': 'mNDV_Busia_2021',
    'region': Busia_shapefile_gee,
    'scale': 10,
    'crs': 'EPSG:4326',
    'maxPixels': 1e13
}


for i in range(0,ndvi_only.size().getInfo()):
    # Get the i-th image from the collection
    image = ee.Image(ndvi_only.toList(ndvi_only.size()).get(i))

    # Get the image name
    image_name = image.get('name').getInfo()

    # Select the NDVI band
    ndvi_band = image.select('mNDVI')

    # Clip the NDVI band using the Busia shapefile geometry
    ndvi_band_clipped = ndvi_band.clip(Busia_shapefile_gee)

    # Set the export parameters for the clipped NDVI band
    export_params['image'] = ndvi_band_clipped
    export_params['description'] = 'mNDVI_' + image_name + '_clipped'
    
    # Export the clipped NDVI band to Google Drive
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()

    # Printing the exporting of images
    print('Exporting ' + image_name + ' clipped...') 

C:\Users\mirmazloumi\Anaconda3\envs\env\lib\site-packages\ipykernel_launcher.py:17: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.


Exporting Image_2021-06-05 clipped...
Exporting Image_2021-06-10 clipped...
Exporting Image_2021-06-20 clipped...
Exporting Image_2021-06-25 clipped...
Exporting Image_2021-06-30 clipped...
Exporting Image_2021-07-05 clipped...
Exporting Image_2021-07-10 clipped...
Exporting Image_2021-07-20 clipped...
Exporting Image_2021-07-25 clipped...
Exporting Image_2021-08-04 clipped...
Exporting Image_2021-08-24 clipped...
Exporting Image_2021-08-29 clipped...
Exporting Image_2021-09-08 clipped...
Exporting Image_2021-09-13 clipped...
Exporting Image_2021-09-23 clipped...
Exporting Image_2021-09-28 clipped...
Exporting Image_2021-10-08 clipped...
Exporting Image_2021-10-13 clipped...
Exporting Image_2021-10-18 clipped...
Exporting Image_2021-10-23 clipped...
Exporting Image_2021-11-17 clipped...
Exporting Image_2021-11-27 clipped...
Exporting Image_2021-12-02 clipped...
Exporting Image_2021-12-07 clipped...
Exporting Image_2021-12-12 clipped...
Exporting Image_2021-12-22 clipped...
Exporting Im

In [15]:
## Displaying the NDVI maps

import geemap
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}

# Create an interactive map that displays each NDVI image as a tile layer
Map = geemap.Map()
Map.centerObject(Busia_shapefile, 10)

for i in range(ndvi_only.size().getInfo()):
    image = ee.Image(ndvi_only.toList(ndvi_only.size()).get(i))
    layer_name = 'mNDVI {}'.format(i + 1)
    Map.addLayer(image.select('mNDVI').clip(Busia_shapefile), vis_params, layer_name)

# Display the map
Map

Map(center=[0.39263292847602566, 34.19357422726364], controls=(WidgetControl(options=['position', 'transparent…